# 此文件用于筛选fpocket的输出结果

In [1]:
import os
import copy

# 函数定义

In [11]:
def is_dibind(file, base=0) -> bool:
    # 用于判断pocket文件中是否同时有蛋白质残基和RNA残基
    # return bool
    with open(file, 'r') as f:
        cont = 1
        for line in f:
            # print(line)
            if line[:4] != 'ATOM': continue
            if cont == 1:
                residue_name_length = len(f.readline()[17:20].strip())
            residue_name = line[17:20].strip()
            cont += 1
            if len(residue_name) == residue_name_length:
                continue
            else:
                return True
    return False

def read_druggability(pockets_info):
    score_dict = dict()
    with open(pockets_info, 'r') as f:
        pockets = f.read().strip().split('\n\n')
        for i, p in enumerate(pockets):
            i += 1
            score_dict[i] = float(p.split('\n')[2].split('\t')[2])
    return score_dict

def fpocket_filter(fpocket_out_folder, low_line=0.5):
    # 此函数用于过滤pocket，利用di_bind函数和分数
    pocket_folder = os.path.join(fpocket_out_folder, 'pockets')
    pockets_info_file = os.path.join(fpocket_out_folder, fpocket_out_folder.split('/')[-1][:-4]+'_info.txt')
    # pockets_file = [os.path.join(pocket_folder, file) for file in os.listdir(pocket_folder) if file[-3:]=='pdb']
    pockets_score = read_druggability(pockets_info_file)
    results = list()
    for i in pockets_score.keys():
        pocket_file_path = os.path.join(pocket_folder,f'pocket{i}_atm.pdb')
        if pockets_score[i] > low_line and is_dibind(pocket_file_path):
            results.append(i)
    return results

def rewrite_out_file(pdb_out_file:str, pockets:list):
    # 此函数用于将不在pockets的pocket删除，并将剩余的pocket重新编号
    with open(pdb_out_file, 'r') as f:
        line_list = f.read().strip().split('\n')
        # out_line = copy.deepcopy(line_list)
        for i, line in enumerate(line_list):
            # print(i)
            line_list[i] = line + '\n'
            line += '\n'
            if line[:6] == 'HETATM' and line[17:20] == 'STP':
                if int(line[22:26]) not in pockets:
                    line_list[i] = ''
                else:
                    line_list[i] = line[:22] + (' '*(4-len(str(pockets.index(int(line[22:26]))+1))) + str(pockets.index(int(line[22:26]))+1)) + line[26:]

    with open(pdb_out_file.replace('.pdb', '_new.pdb'), 'w') as f:
        f.writelines(line_list)
    # rewrite pml
    with open(pdb_out_file.replace('_out.pdb', '.pml'), 'r') as f:
        lines = f.readlines()
    with open(pdb_out_file.replace('_out.pdb', '_new.pml'), 'w') as f:
        pml_name = pdb_out_file.split('/')[-1].replace('.pdb', '_new.pdb')
        lines[1] = f'load {pml_name}\n'
        f.writelines(lines)
        

# main函数

In [12]:
if __name__ == "__main__":
    # fpocket输出文件夹，例："E:\tmp\fpocket\6v5b_out"
    fpocket_out_folder = r'/mnt/e/tmp/fpocket/6v5c_out'
    pdb_out_file = os.path.join(fpocket_out_folder, fpocket_out_folder.split('/')[-1]+'.pdb')
    # pml_out_file = os.path.join(fpocket_out_folder, fpocket_out_folder.split('/')[-1].replace('_out.pdb', '.pml'))
    pockets = fpocket_filter(fpocket_out_folder, low_line=0.1)
    # print(pockets)
    rewrite_out_file(pdb_out_file, pockets)

In [13]:
read_druggability(fpocket_out_folder+'/6v5c_info.txt')

{1: 0.0,
 2: 0.094,
 3: 0.015,
 4: 0.002,
 5: 0.002,
 6: 0.012,
 7: 0.002,
 8: 0.004,
 9: 0.129,
 10: 0.001,
 11: 0.001,
 12: 0.004,
 13: 0.014,
 14: 0.001,
 15: 0.007,
 16: 0.0,
 17: 0.942,
 18: 0.0,
 19: 0.001,
 20: 0.001,
 21: 0.0,
 22: 0.001,
 23: 0.001,
 24: 0.002,
 25: 0.002,
 26: 0.001,
 27: 0.0,
 28: 0.001,
 29: 0.001,
 30: 0.04,
 31: 0.321,
 32: 0.002,
 33: 0.0,
 34: 0.0,
 35: 0.004,
 36: 0.006,
 37: 0.002,
 38: 0.001,
 39: 0.001,
 40: 0.0,
 41: 0.0,
 42: 0.001,
 43: 0.001,
 44: 0.002,
 45: 0.0,
 46: 0.001,
 47: 0.004,
 48: 0.001,
 49: 0.001,
 50: 0.001,
 51: 0.001,
 52: 0.008,
 53: 0.002,
 54: 0.0,
 55: 0.001,
 56: 0.001,
 57: 0.0,
 58: 0.0,
 59: 0.012,
 60: 0.004,
 61: 0.0,
 62: 0.002,
 63: 0.036,
 64: 0.0,
 65: 0.001,
 66: 0.024,
 67: 0.003,
 68: 0.004,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.913,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.001,
 79: 0.0,
 80: 0.0,
 81: 0.001,
 82: 0.017,
 83: 0.001,
 84: 0.003,
 85: 0.005,
 86: 0.003,
 87: 0.885,
 88: 0.001,
 89:

# 输出文件

In [37]:
19, 58